In [29]:
import minitorch
import pytest
from minitorch import History

class FunctionBase:
    """
    A function that can act on :class:`Variable` arguments to
    produce a :class:`Variable` output, while tracking the internal history.

    Call by :func:`FunctionBase.apply`.

    """

    @staticmethod
    def variable(raw, history):
        # Implement by children class.
        raise NotImplementedError()

    @classmethod
    def apply(cls, *vals):
        """
        Apply is called by the user to run the Function.
        Internally it does three things:

        a) Creates a Context for the function call.
        b) Calls forward to run the function.
        c) Attaches the Context to the History of the new variable.

        There is a bit of internal complexity in our implementation
        to handle both scalars and tensors.

        Args:
            vals (list of Variables or constants) : The arguments to forward

        Returns:
            `Variable` : The new variable produced

        """
        # Go through the variables to see if any needs grad.
        raw_vals = []
        need_grad = False
        for v in vals:
            if isinstance(v, Variable):
                if v.history is not None:
                    need_grad = True
                v.used += 1
                raw_vals.append(v.get_data())
            else:
                raw_vals.append(v)

        # Create the context.
        ctx = Context(not need_grad)

        # Call forward with the variables.
        c = cls.forward(ctx, *raw_vals)
        assert isinstance(c, cls.data_type), "Expected return typ %s got %s" % (
            cls.data_type,
            type(c),
        )

        # Create a new variable from the result with a new history.
        back = None
        if need_grad:
            back = History(cls, ctx, vals)
        return cls.variable(cls.data(c), back)

    @classmethod
    def chain_rule(cls, ctx, inputs, d_output):
        """
        Implement the derivative chain-rule.

        Args:
            ctx (:class:`Context`) : The context from running forward
            inputs (list of args) : The args that were passed to :func:`FunctionBase.apply` (e.g. :math:`x, y`)
            d_output (number) : The `d_output` value in the chain rule.

        Returns:
            list of (`Variable`, number) : A list of non-constant variables with their derivatives
            (see `is_constant` to remove unneeded variables)

        """
        # Tip: Note when implementing this function that
        # cls.backward may return either a value or a tuple.

        print(cls, ctx, inputs, d_output)
        
        return []


class ScalarFunction(FunctionBase):
    """
    A wrapper for a mathematical function that processes and produces
    Scalar variables.

    This is a static class and is never instantiated. We use `class`
    here to group together the `forward` and `backward` code.
    """

    @staticmethod
    def forward(ctx, *inputs):
        r"""
        Forward call, compute :math:`f(x_0 \ldots x_{n-1})`.

        Args:
            ctx (:class:`Context`): A container object to save
                                    any information that may be needed
                                    for the call to backward.
            *inputs (list of floats): n-float values :math:`x_0 \ldots x_{n-1}`.

        Should return float the computation of the function :math:`f`.
        """
        pass  # pragma: no cover

    @staticmethod
    def backward(ctx, d_out):
        r"""
        Backward call, computes :math:`f'_{x_i}(x_0 \ldots x_{n-1}) \times d_{out}`.

        Args:
            ctx (Context): A container object holding any information saved during in the corresponding `forward` call.
            d_out (float): :math:`d_out` term in the chain rule.

        Should return the computation of the derivative function
        :math:`f'_{x_i}` for each input :math:`x_i` times `d_out`.

        """
        pass  # pragma: no cover

    # Checks.
    variable = minitorch.scalar.Scalar
    data_type = float

    @staticmethod
    def data(a):
        return a


# ## Task 1.3 - Tests for the autodifferentiation machinery.

# Simple sanity check and debugging tests.


class Function1(ScalarFunction):
    @staticmethod
    def forward(ctx, x, y):
        ":math:`f(x, y) = x + y + 10`"
        return x + y + 10

    @staticmethod
    def backward(ctx, d_output):
        "Derivatives are :math:`f'_x(x, y) = 1` and :math:`f'_y(x, y) = 1`"
        return d_output, d_output


class Function2(ScalarFunction):
    @staticmethod
    def forward(ctx, x, y):
        ":math:`f(x, y) = x \timex y + x`"
        ctx.save_for_backward(x, y)
        return x * y + x

    @staticmethod
    def backward(ctx, d_output):
        "Derivatives are :math:`f'_x(x, y) = y + 1` and :math:`f'_y(x, y) = x`"
        x, y = ctx.saved_values
        return d_output * (y + 1), d_output * x


# Checks for the chain rule function.


# @pytest.mark.task1_3
# def test_chain_rule1():
#     "Check that constants are ignored."
#     constant = minitorch.Variable(None)
#     back = Function1.chain_rule(ctx=None, inputs=[constant, constant], d_output=5)
#     assert len(list(back)) == 0


# @pytest.mark.task1_3
# def test_chain_rule2():
#     "Check that constrants are ignored and variables get derivatives."
#     var = minitorch.Variable(History())
#     constant = minitorch.Variable(None)
#     back = Function1.chain_rule(ctx=None, inputs=[var, constant], d_output=5)
#     back = list(back)
#     assert len(back) == 1
#     variable, deriv = back[0]
#     assert variable.name == var.name
#     assert deriv == 5


# @pytest.mark.task1_3
# def test_chain_rule3():
#     "Check that constrants are ignored and variables get derivatives."
#     constant = 10
#     var = minitorch.Scalar(5)

#     ctx = minitorch.Context()
#     Function2.forward(ctx, constant, var.data)

#     back = Function2.chain_rule(ctx=ctx, inputs=[constant, var], d_output=5)
#     back = list(back)
#     assert len(back) == 1
#     variable, deriv = back[0]
#     assert variable.name == var.name
#     assert deriv == 5 * 10

# @pytest.mark.task1_3
# def test_chain_rule4():
#     var1 = minitorch.Scalar(5)
#     var2 = minitorch.Scalar(10)

#     ctx = minitorch.Context()
#     Function2.forward(ctx, var1.data, var2.data)

#     back = Function2.chain_rule(ctx=ctx, inputs=[var1, var2], d_output=5)
#     back = list(back)
#     assert len(back) == 2
#     variable, deriv = back[0]
#     assert variable.name == var1.name
#     assert deriv == 5 * (10 + 1)
#     variable, deriv = back[1]
#     assert variable.name == var2.name
#     assert deriv == 5 * 5

constant = minitorch.Variable(None)
back = Function1.chain_rule(ctx=None, inputs=[constant, constant], d_output=5)
# assert len(list(back)) == 0

back

<class '__main__.Function1'> None [<minitorch.autodiff.Variable object at 0x127395b80>, <minitorch.autodiff.Variable object at 0x127395b80>] 5


[]

In [4]:
def gradients(func, vars_list, eps=1e-4):
    partial_derivatives = []
    base_func_eval = func(*vars_list)

    for idx in range(len(vars_list)):
        tweaked_vars = vars_list[:]
        tweaked_vars[idx] += eps
        tweaked_func_eval = func(*tweaked_vars)
        derivative = (tweaked_func_eval - base_func_eval) / eps
        partial_derivatives.append(derivative)
    
    return partial_derivatives

In [8]:
def f(x,y):
    return x*x*y + y + 2

def df(x, y):
    return gradients(f, [x, y])

In [9]:
df(3, 4)

[24.000400000048216, 10.000000000047748]

In [10]:
def dfdx(x, y):
    return gradients(f, [x, y])[0]

def dfdy(x, y):
    return gradients(f, [x, y])[1]

dfdx(3, 4), dfdy(3, 4)

(24.000400000048216, 10.000000000047748)

In [12]:
def d2f(x, y):
    return [gradients(dfdx, [x, y]), gradients(dfdy, [x, y])]

d2f(3, 4)

[[7.999999951380232, 6.000099261882497],
 [6.000099261882497, -1.4210854715202004e-06]]

In [13]:
class Const:
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        return self.value

    def __str__(self):
        return str(self.value)


class Var:
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name

    def evaluate(self):
        return self.value

    def __str__(self):
        return str(self.value)


class BinaryOperator:
    def __init__(self, a, b):
        self.a = a
        self.b = b


class Add(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() + self.b.evaluate()

    def __str__(self):
        return f"{self.a} + {self.b}"


class Mul(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() * self.b.evaluate()

    def __str__(self):
        return f"{self.a} * {self.b}"


In [15]:
x = Var("x")
y = Var("y")
f = Add(Mul(Mul(x, x), y), Add(y, Const(2)))

In [18]:
x.value = 3
y.value = 4
f.evaluate()

42

In [19]:
from math import sin

def z(x):
    return sin(x ** 2)

gradients(z, [3])

[-5.46761419430053]

In [23]:
Const.gradient = lambda self, var: Const(0)
Var.gradient = lambda self, var: Const(1) if self is var else Const(0)
Add.gradient = lambda self, var: Add(self.a.gradient(var), self.b.gradient(var))
Mul.gradient = lambda self, var: Add(Mul(self.a, self.b.gradient(var)), Mul(self.a.gradient(var), self.b))

x = Var(name="x", init_value=3.)
y = Var(name="y", init_value=4.)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2)))

dfdx = f.gradient(x)
dfdy = f.gradient(y)

In [24]:
dfdx.evaluate(), dfdy.evaluate()

(24.0, 10.0)

In [28]:
[g.evaluate() for g in [dfdx.gradient(x), dfdx.gradient(y), dfdy.gradient(x), dfdy.gradient(y)]]

[8.0, 6.0, 6.0, 0.0]

In [120]:
class Const(): # f(x,y) = x²y + y + 2
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        print('const eval', self.value)
        return self.value
    
    def backpropagate(self, gradient):
        print('const back', gradient)
        pass

    def __str__(self):
        return str(self.value)

class Var: # f(x,y) = x²y + y + 2
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
        self.gradient = 0

    def evaluate(self):
        print('var eval', self.value)
        return self.value

    def backpropagate(self, gradient):
        print('var back', self.gradient, gradient)
        self.gradient += gradient
    
    def __str__(self):
        return self.name

class BinaryOperator(): # f(x,y) = x²y + y + 2
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
class Add(BinaryOperator): # f(x,y) = x²y + y + 2
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        print('add eval', self.a.evaluate(), self.b.evaluate())
        return self.value
    def backpropagate(self, gradient):
        print('add back', self.a, self.b, gradient)
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator): # f(x,y) = x²y + y + 2
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        print('mul eval', self.a.evaluate(), self.b.evaluate())
        return self.value
    def backpropagate(self, gradient):
        print('mul back', self.a, self.b, gradient)
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return "{} * {}".format(self.a, self.b)

In [ ]:
x = Var("x", init_value=3)
y = Var("y", init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2)))

result = f.evaluate()
result

In [132]:
f.backpropagate(1.0)

add back x * x * y y + 2 1.0
mul back x * x y 1.0
mul back x x 4.0
var back 0 12.0
var back 12.0 12.0
var back 0 9.0
add back y 2 1.0
var back 9.0 1.0
const back 1.0


In [123]:
print(f)

x * x * y + y + 2


In [124]:
x.gradient

24.0

In [125]:
y.gradient

10.0

In [270]:
import numpy as np
from functools import reduce
from operator import mul

ordinal = 1  # (1, 0)
shape = (4, 1, 2)
# array = np.array([[[1, 1, 1]], [[1, 1, 1]], [[0, 1, 1]], [[0, 1, 1]]])
out_index = []

for i, s in enumerate(shape):
    product = reduce(mul, shape[i:], 1)
    divisor = product / s
    index = int(ordinal // divisor)
    
    ordinal -= divisor * index
    out_index.append(index)

    print(f"{ i = }, { s = }, { product = }, { divisor = } { ordinal = }")

out_index

 i = 0,  s = 4,  product = 8,  divisor = 2.0  ordinal = 1.0
 i = 1,  s = 1,  product = 2,  divisor = 2.0  ordinal = 1.0
 i = 2,  s = 2,  product = 2,  divisor = 1.0  ordinal = 0.0


[0, 0, 1]

In [205]:
min(1, 2)

1

In [177]:

ordinal = 3
a, b, c = array.shape # (4, 1, 3)
print(ordinal // a)

ordinal /=  a
ordinal # 3

print(12 // b)
ordinal /= b
ordinal # 3

c - 1

0
12


2

In [ ]:
or

In [12]:
# 3 // 2 -> 1
# 1 // 3 -> 0

1